# 1. Import Libraries

In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from glob import glob
from sklearn.model_selection import train_test_split

# 2. Define Image Size and Batch Size:

In [20]:
IMAGE_SIZE = 224 # define the image size for all images (height and Width = 224 X 224)
BATCH_SIZE = 32  # At the time of training 64 images will be there at a time

# 3. Set Folder Paths:

In [21]:
base_dir = "/kaggle/input/tomatoleaf/tomato"
train_dir = "/kaggle/input/tomatoleaf/tomato/train"
validation_dir = "/kaggle/input/tomatoleaf/tomato/val"

# 4. Get the Number of Classes from Folder Names:

In [22]:
folders = glob(train_dir + '/*')
print(len(folders))
num_classes = len(folders)

10


# 5. Load DenseNet121 Model:

In [23]:
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 6. Modify the output layer

In [24]:
x = densenet_model.output
x = GlobalAveragePooling2D()(x)

# Dense layers with dropout regularization
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)

predictions = Dense(num_classes, activation='sigmoid')(x)

# 7. Freeze Convolutional Layers (Optional Fine-Tuning):

In [25]:
# Unfreeze the last few layers of the base model
for layer in densenet_model.layers[-10:]:
    layer.trainable = True

# 8. Create a new model with the new output layer

In [26]:
model = Model(inputs=densenet_model.input, outputs=predictions)
# model.summary()

# 9. Compile the Model (Initial Training):

In [27]:
# compile the model
rmsprop_optimizer = RMSprop(learning_rate=0.0001, rho=0.9)
model.compile(optimizer=rmsprop_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 10. Data Augmentation for Training and Validation:

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split = 0.2  # 20% for validation
)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2 # 20% for validation
)
# for acutual val folder data
test_datagen = ImageDataGenerator(rescale=1./255)

# 11. Prepare Data Generators:

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
)

test_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical'
)

Found 10000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


# 11. Early Stopping and Learning Rate Reduction:

In [12]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, min_lr=0.0001)
model_checkpoint = ModelCheckpoint('tomato_densenet121_sigmoid_model_update1.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)

# 12. Train the Model (Initial):

In [13]:
epoch = 20
history = model.fit(
    train_generator,
    epochs=epoch,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1715930335.231185      85 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 798ms/step - accuracy: 0.3496 - loss: 1.8225
Epoch 1: val_loss improved from inf to 0.45725, saving model to tomato_densenet121_sigmoid_model_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 438s 929ms/step - accuracy: 0.3504 - loss: 1.8203 - val_accuracy: 0.8785 - val_loss: 0.4572 - learning_rate: 1.0000e-04
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.9283 - loss: 0.2440
Epoch 2: val_loss improved from 0.45725 to 0.18744, saving model to tomato_densenet121_sigmoid_model_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 143s 449ms/step - accuracy: 0.9284 - loss: 0.2440 - val_accuracy: 0.9450 - val_loss: 0.1874 - learning_rate: 1.0000e-04
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - accuracy: 0.9518 - loss: 0.1673
Epoch 3: val_loss improved from 0.18744 to 0.09894, saving model to tomato_densenet121_sigmoid_model_update1.weights.h5
313/313 ━━━━━━━━━━━━━━━━━━━━ 141s 442ms/step - accuracy: 0.9518 - loss: 0.1673 - val_accuracy

# 13. Evaluate the Model after Initial Training:

In [14]:
score = model.evaluate(test_generator)
print("Test loss (initial):", score[0])
print("Test accuracy (initial):", score[1])

32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 629ms/step - accuracy: 0.9725 - loss: 0.0926
Test loss (initial): 0.13394126296043396
Test accuracy (initial): 0.9729999899864197


# 14. Get the classification Report

In [15]:
import numpy as np
from sklearn.metrics import classification_report
class_labels = ['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 
                'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
                'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
                'Tomato___healthy']

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_labels = test_generator.classes
report = classification_report(true_labels, predicted_classes, target_names=class_labels)
print(report)

32/32 ━━━━━━━━━━━━━━━━━━━━ 22s 397ms/step
                                               precision    recall  f1-score   support

                      Tomato___Bacterial_spot       0.99      0.96      0.97       100
                        Tomato___Early_blight       0.99      0.97      0.98       100
                         Tomato___Late_blight       0.96      1.00      0.98       100
                           Tomato___Leaf_Mold       1.00      0.98      0.99       100
                  Tomato___Septoria_leaf_spot       0.91      1.00      0.95       100
Tomato___Spider_mites Two-spotted_spider_mite       0.97      0.96      0.96       100
                         Tomato___Target_Spot       1.00      0.88      0.94       100
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       0.99      0.98      0.98       100
                 Tomato___Tomato_mosaic_virus       0.96      1.00      0.98       100
                             Tomato___healthy       0.97      1.00      0.99       100


# 15. Update the weights of the model

In [17]:
model.load_weights('/kaggle/working/tomato_densenet121_sigmoid_model_update1.weights.h5')